In [1]:
import os
import sys
from pathlib import Path

# Move up to project root (adjust depending on your folder structure)
PROJECT_ROOT = Path(os.getcwd()).parents[0]  # research → project root
os.chdir(PROJECT_ROOT)
sys.path.append(str(PROJECT_ROOT))

print("Running from:", os.getcwd())

Running from: c:\Users\USER\Desktop\001_MLops_project\001_Datascience_project


In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float # hyperparameter for elastic net ML algo
    l1_ratio: float # hyperparameter for elastic net ML algo
    target_column: str

In [14]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        
        # read the yaml file stated below
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        # read then extract the dir name from the config file just read
        # this allows us creat the artifacts/ folder (where outputs will be stored)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_cofig(self) -> ModelTrainingConfig:
        config=self.config.model_training
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        # create saved moder dir
        create_directories([config.root_dir])


        model_trainer_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config





In [24]:
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import pandas as pd
import joblib

In [25]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config


    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop(self.config.target_column, axis=1)
        test_x = test_data.drop(self.config.target_column, axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [26]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_cofig()
    model_trainer =  ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-07-09 15:11:51,227: INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-09 15:11:51,234: INFO: yaml file: params.yaml loaded successfully]
[2025-07-09 15:11:51,246: INFO: yaml file: schema.yaml loaded successfully]
[2025-07-09 15:11:51,250: INFO: create directory at: artifacts]
[2025-07-09 15:11:51,255: INFO: create directory at: artifacts/model_training]
